In [1]:
!pip install git+https://github.com/sitfoxfly/gomoku-ai

  Running command git clone --filter=blob:none --quiet https://github.com/sitfoxfly/gomoku-ai 'C:\Users\ALEXANDER WILLY JOHA\AppData\Local\Temp\pip-req-build-tat_giyq'



  Cloning https://github.com/sitfoxfly/gomoku-ai to c:\users\alexander willy joha\appdata\local\temp\pip-req-build-tat_giyq
  Resolved https://github.com/sitfoxfly/gomoku-ai to commit 13efdaa4ad34f4c42fe600998528c6d2c09e63de
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


In [2]:
# Import the necessary modules
import random
from typing import Tuple
import re
import json

# Import the game framework components
from gomoku.agents.base import Agent
from gomoku.core.models import GameState, Player
from gomoku.arena import GomokuArena
from gomoku.utils import ColorBoardFormatter
from gomoku.llm.openai_client import OpenAIGomokuClient
from gomoku.core.game_logic import GomokuGame

from dotenv import load_dotenv
load_dotenv()

True

# Import Agents

In [4]:
# from <filename-without-py> import <class-name>
from GomokuAgent import GomokuAgent
from GomokuAgentV2 import GomokuAgentV2
from Mark2 import Mark2Agent
from MarkIV import Mark4Agent
from MarkV import Mark5Agent
from PromptEngineerV2 import PromptEngineerV2
from PromptEngineerV3 import PromptEngineerV3

# Arena Definition

In [6]:
# Setup the competition arena
print("🏟️  Setting up the Competition Arena!")
print("=" * 50)

board_size = 8  # Smaller board for faster games
formatter = ColorBoardFormatter(board_size)
arena = GomokuArena(board_size=board_size, formatter=formatter)

print(f"⚙️  Arena Configuration:")
print(f"   📏 Board size: {board_size}x{board_size}")
print(f"   🎨 Visualization: Color-coded board")

agents = {
    # 'GomokuAgent': GomokuAgent("GomokuAgent"),
    # 'GomokuAgentV2': GomokuAgentV2("GomokuAgentV2"),
    # 'Mark2Agent': Mark2Agent("Mark2Agent"),
    'Mark4Agent': Mark4Agent("Mark4Agent"),
    'Mark5Agent':Mark5Agent("Mark5Agent"),
    # 'PromptEngineerV2': PromptEngineerV2("PromptEngineerV2"),
    # 'PromptEngineerV3': PromptEngineerV3("PromptEngineerV3")
}

print(f"\n🤖 Competitors Ready:")
for name, agent in agents.items():
    print(f"   • {agent.agent_id} ({name})")

print("\n🎮 Ready for competition!")

🏟️  Setting up the Competition Arena!
⚙️  Arena Configuration:
   📏 Board size: 8x8
   🎨 Visualization: Color-coded board

🤖 Competitors Ready:
   • Mark4Agent (Mark4Agent)
   • Mark5Agent (Mark5Agent)

🎮 Ready for competition!


In [7]:
import sys
import io
import contextlib
import time
from tqdm import tqdm
import os

class Tee:
    def __init__(self, *streams):
        self.streams = streams
    def write(self, data):
        for s in self.streams:
            s.write(data)
            s.flush()
    def flush(self):
        for s in self.streams:
            s.flush()

async def compete(agent1, agent2):
    buffer = io.StringIO()
    tee = Tee(sys.stdout, buffer)  # Send output to console + buffer

    os.makedirs('./logs', exist_ok=True)
    
    with contextlib.redirect_stdout(tee):
        print(f"COMPETITION: {agent1} vs {agent2}")
        print("=" * 60)

        game_result = await arena.run_game(
            agents[agent1],
            agents[agent2],
            verbose=True
        )


        print("\n" + "=" * 60)
        print("GAME RESULTS")
        print("=" * 60)
        print(f"Winner: {game_result['winner']}")
        print(f"Total moves: {game_result['moves']}")
        print(f"Game duration: {game_result.get('total_time', 0):.2f} seconds")
        print(f"Victory condition: {game_result['reason']}")

        if game_result.get('winning_sequence'):
            print(f"Winning sequence: {len(game_result['winning_sequence'])} positions")

        print("\nFinal Board with Winning Sequence:")
        final_display = arena.draw_board_with_winning_sequence(
            game_result['final_board'],
            game_result.get('winning_sequence', [])
        )
        print(final_display)
    
    with open(f"./logs/{agent1}_x_{agent2}_log.txt", 'w', encoding='utf-8') as f:
        f.write(buffer.getvalue())
        
    return game_result['winner']

In [8]:
# await compete('Mark4Agent','PromptEngineerV2')

# Create Leaderboard

In [10]:
# Setup Leaderboard
leaderboard = {}
for a in agents.keys():
    if a not in leaderboard:
        leaderboard[a] = [0, 0, 0, 0, 0] # points, num_played, win, draw, loss

def update_leaderboard(leaderboard, winner, p1_idx, p2_idx, names):
    # update num_played
    leaderboard[ names[p1_idx] ][1] += 1
    leaderboard[ names[p2_idx] ][1] += 1

    if winner == None: # Draw
        # update num of draw & points (+1) for p1 and p2
        leaderboard[ names[p1_idx] ][3] += 1
        leaderboard[ names[p2_idx] ][3] += 1
        
        leaderboard[ names[p1_idx] ][0] += 1
        leaderboard[ names[p2_idx] ][0] += 1
    else:
        if winner.strip() == names[p1_idx].strip():
            # update num of win & points (+3) for p1
            leaderboard[ names[p1_idx] ][2] += 1
            leaderboard[ names[p1_idx] ][0] += 3
            
            # update num of loss for p2
            leaderboard[ names[p2_idx] ][4] += 1
    
        elif winner.strip() == names[p2_idx].strip():
            # update num of win & points (+3) for p2
            leaderboard[ names[p2_idx] ][2] += 1
            leaderboard[ names[p2_idx] ][0] += 3
            
            # update num of loss for p1
            leaderboard[ names[p1_idx] ][4] += 1

    return leaderboard
leaderboard

{'Mark4Agent': [0, 0, 0, 0, 0], 'Mark5Agent': [0, 0, 0, 0, 0]}

# Play

In [12]:
names = list(agents.keys())

for p1_idx in range( len(names) ):
    for p2_idx in range(p1_idx+1, len(names)):
        # first game
        print(names[p1_idx] , 'vs',  names[p2_idx] )
        winner = await compete(names[p1_idx], names[p2_idx])
        leaderboard = update_leaderboard(leaderboard, winner, p1_idx, p2_idx, names)

        

        # second game
        print(names[p2_idx] , 'vs',  names[p1_idx] )
        winner = await compete(names[p2_idx], names[p1_idx])
        leaderboard = update_leaderboard(leaderboard, winner, p1_idx, p2_idx, names)

        
        print('='*30, "NEXT GAME", '='*30)

Mark4Agent vs Mark5Agent
COMPETITION: Mark4Agent vs Mark5Agent

BLACK's turn (Mark4Agent)
    0  1  2  3  4  5  6  7 
 0  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  . 
 3  .  .  .  .  .  .  .  . 
 4  .  .  .  .  .  .  .  . 
 5  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  . 

Move: (4, 4) in 3.59s

WHITE's turn (Mark5Agent)
    0  1  2  3  4  5  6  7 
 0  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  . 
 3  .  .  .  .  .  .  .  . 
 4  .  .  .  .  X  .  .  . 
 5  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  . 

Move: (3, 3) in 7.13s

BLACK's turn (Mark4Agent)
    0  1  2  3  4  5  6  7 
 0  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  . 
 3  .  .  .  O  .  .  .  . 
 4  .  .  .  .  X  .  .  . 
 5  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  . 

Move: (4, 3) in 2.98s

WHITE's turn (Mark5Agent)
    

In [13]:
sorted_leaderboard = sorted(
    leaderboard.items(),
    key=lambda x: x[1][0],
    reverse=True
)

In [14]:
# Header
headers = ["Agent", "Total Points", "Played", "Win", "Draw", "Loss"]
print(f"{headers[0]:>20} {headers[1]:>13} {headers[2]:>7} {headers[3]:>5} {headers[4]:>5} {headers[5]:>5}")
print("-" * 60)

# Rows
for agent, stats in sorted_leaderboard:
    print(f"{agent:>20} {stats[0]:>13} {stats[1]:>7} {stats[2]:>5} {stats[3]:>5} {stats[4]:>5}")

               Agent  Total Points  Played   Win  Draw  Loss
------------------------------------------------------------
          Mark4Agent             6       2     2     0     0
          Mark5Agent             0       2     0     0     2
